ISSUE CASE STUDY

I. BŁĘDNE RZUTOWANIE TYPÓW ZMIENNYCH 
    java.sql.SQLException: Bad value for type BigDecimal : 2A

ROZPOZNANIE:

    SqLite wyświetla schemat tabeli 'seats' w następujący sposób:
        sqlite> pragma table_info(seats);
        0|aircraft_code|character(3)|1||0
        1|seat_no|character varying(4)|1||0
        2|fare_conditions|character varying(10)|1||0

    natomiast PySpark:
       root
        |-- aircraft_code: string (nullable = true)
        |-- seat_no: decimal(4,0) (nullable = true)
        |-- fare_conditions: decimal(10,0) (nullable = true)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

ROZWIĄZANIE 1:
    

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType

# Definiujemy poprawny schemat ręcznie
schema = StructType([
    StructField("aircraft_code", StringType(), True),
    StructField("seat_no", StringType(), True),  # Wymuszamy STRING, a nie DECIMAL
    StructField("fare_conditions", StringType(), True)
])

# Odczytujemy tabelę z wymuszonym schematem
df = spark.read.format("jdbc") \
    .option("url", "jdbc:sqlite:/ścieżka/do/bazy.db") \
    .option("dbtable", "seats") \
    .option("driver", "org.sqlite.JDBC") \
    .schema(schema) \
    .load()

df.printSchema()
df.show()

BRAK ROZWIĄZANIA. NIE MOŻNA PRZYPISAĆ SCHEMATU DO ISTNIEJĄCEJ TABELI.

ROZWIĄZANIE 2:

In [ ]:
df_fixed = dfs["seats"].withColumn("seat_no", col("seat_no").cast("string")) \
            .withColumn("fare_conditions", col("fare_conditions").cast("string")).show(5, False)

Utworzenie nowego DataFrame ze zmienionym typem danych niestety powoduje znowu wystąpienie tego samego błędu

ROZWIĄZANIE 3:

Zmiana wersji sqlite-jdbc z 3.49.1.0 na 3.34.0 nie wniosła nic.

ROZWIĄZANIE 4:

Utworzenie nowej bazy danych z podstawowymi typami dancyh dla SQLite jak TEXT i INTEGER rozwiązało kłopoty.